In [13]:
%matplotlib inline
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from matplotlib import rc
from functools import partial
rc('animation', html='jshtml')

In [14]:
# Define a function to generate a random weak learner
def get_weak_learner(x, r):
  h = np.random.normal(0, 1, len(r))
  h = h / np.linalg.norm(h)
  inner = np.inner(h, r)
  if inner < 0:
      return h
  else:
      return -1 * h

# Sum of Squared Error
def calc_loss(cur, y):
  return 0.5 * sum((v1 - v2)**2 for v1, v2 in zip(cur, y))

# Perform one Boosting step
def boost(x,y,H,alpha):
  r = H - y
  h = get_weak_learner(x,r)
  H_new = H + alpha*h
  loss = calc_loss(H_new,y)
  return H_new, loss

In [15]:
# Functions for the plotting
def generate_axes(fig):
  gridspec = fig.add_gridspec(nrows=2, ncols=9)
  ax = fig.add_subplot(gridspec[0:2, 0:4], projection = '3d')
  ax2 = fig.add_subplot(gridspec[1:2, 5:9])
  return ax,ax2

def init_plot(ax,ax2,y,H):
  ax.plot(y[0],y[1],y[2],'*b',label = 'Target')
  ax.text(y[0]-0.5,y[1]+0.05,y[2]+0.1,f'$y=$({y[0]},{y[1]},{y[2]})',size = 8)
  ax.plot(H[0],H[1],H[2],'.r', label = 'Prediction')
  line, = ax.plot([], [], [], '-r', linewidth = 2)
  H_text = ax.text(*(H-0.1),f'$H_0=$({H[2]},{H[1]},{H[2]})',size = 8)
  ax.legend()
  mn, mx = -1, 1
  ticks = np.linspace(mn,mx,5)
  ax.set(xlim = [mn,mx], ylim = [mn, mx], zlim = [mn, mx], 
      xticks = ticks, yticks = ticks, zticks = ticks,
      xlabel = '$y_1$', ylabel = '$y_2$', zlabel = '$y_3$')

  line2, = ax2.plot(range(N),L, '-r', linewidth = 2, label = 'SSE')
  title = ax2.set_title(f'Loss after 0 Iterations: {loss:.5f}')
  ax2.legend()
  ax2.set(xlabel = 'Iteration', ylabel = 'SSE')
  return ax, ax2, line, H_text, line2, title

def init_plot_fake(art):
  line, line2, title, H_text = art
  return art

def update(frame, x, y, H ,L, art):
  line, line2, title, H_text = art
  H_i = H[frame]
  
  # This is where the Boosting happen
  H_new, loss = boost(x,y,H_i,alpha)

  H.append(H_new)
  L[frame] = loss
  Xs.append(H[frame][0])
  Ys.append(H[frame][1])
  Zs.append(H[frame][2])
  line.set_data(Xs, Ys)
  line.set_3d_properties(Zs)
  line2.set_data(range(len(L)),L)
  title.set_text(f'Loss after {frame} Iterations: {loss:.5f}')
  H_text.set_text(f'$H_{{{frame}}}=$({H[frame][0]:.2f},{H[frame][1]:.2f},{H[frame][2]:.2f})')
  H_text.set(position_3d = H[frame])
  return art

In [ ]:
# Set the seed for the random number generator to ensure reproducibility
np.random.seed(50)

# Set the learning rate and number of rounds
alpha = 0.05
N = 101

# Set the number of dimensions and generate a random target vector
p = 3
x = np.random.rand(p)
y = np.array((1,-1,-1)+ (-1, ) * (p-3))

# Initialize the hypothesis and loss
H = [np.array((0, ) * p),]
Xs, Ys, Zs = [H[0][0],],[H[0][1],],[H[0][2],]
L = np.zeros(N)
loss = calc_loss(H[0], y)
L[0] = loss

# Set up the 3D plot for visualization
fig = plt.figure(figsize = plt.figaspect(0.5))
ax, ax2 = generate_axes(fig)
ax, ax2, line, H_text, line2, title = init_plot(ax,ax2, y, H[0])
art = [line, line2, title, H_text]

# Run Bossting and generate animation
anim = animation.FuncAnimation(fig, partial(update, x = x, y = y, H = H, L = L, art = art), 
                               interval=300, frames = N, blit=True, 
                               init_func=partial(init_plot_fake, art))
plt.close(fig)

anim

# Save to file
# FFwriter = animation.FFMpegWriter(fps=10)
# anim.save('animation.mp4', writer = FFwriter)